In [1]:
import numpy as np
import cv2
import IPython
from IPython.display import clear_output

import sys 
import os
sys.path.append(os.path.abspath("../"))
from module.Segmentor import Segmentor

import time

In [2]:
# TODO : Save output to file

filename = '../cards/test.mp4'

capture = cv2.VideoCapture(filename)

while capture.isOpened():
    ret, frame = capture.read()
    
    if not ret:
        print("End of stream? Exiting...")
        break
        
    # Method 1
    segmentor = Segmentor(
        min_p=4,
        max_p=400,
        min_a=1000,
        max_a=20000,
        suit_x_ratio=2 / 7,
        suit_y_ratio=1 / 5,
    )
    
    st = time.time()
    cardSuits = segmentor.seg(inputImage)
    en = time.time()
    print("time:", en - st)
    
    for i in range(len(cardSuits)):
        cv2.imshow("card" + str(i), cardSuits[i])
    
    
    outputVideoFrame = np.hstack((frame,frame,frame))
    # Encode image as jpg numpy array
    _, buf = cv2.imencode(".jpg", outputVideoFrame)
    # Draw result
    IPython.display.display(IPython.display.Image(data=buf))
        
    clear_output(wait=True) 
    
    if cv2.waitKey(1) == ord('q'):
        break

time: 0.0024480819702148438


NameError: name 'i' is not defined